In [31]:
from math import sqrt

import numpy
import pandas
from scipy import stats

# 1

In [2]:
water = pandas.read_table('water.txt', sep='\t')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [10]:
correlation = water["hardness"].corr(water["mortality"])
print(f'{correlation:.4f}')

-0.6548


# 2

In [11]:
correlation = water["hardness"].corr(water["mortality"], method="spearman")
print(f'{correlation:.4f}')

-0.6317


# 3

In [23]:
north = water[water['location'] == 'North']
north_correlation = north['hardness'].corr(north['mortality'])
print(f'North: {north_correlation:.4f}')

south = water[water['location'] == 'South']
south_correlation = south['hardness'].corr(south['mortality'])
print(f'South: {south_correlation:.4f}')

print(f'Answer: {min(north_correlation, south_correlation, key=abs):.4f}')

North: -0.3686
South: -0.6022
Answer: -0.3686


# 4

In [28]:
a = 203
b = 239
c = 718
d = 515

correlation = (a * d - b * c) / sqrt((a + b) * (a + c) * (b + d) * (c + d))
print(f'{correlation:.3f}')

-0.109


# 5

In [37]:
chi2, p, dof, expected = stats.chi2_contingency(numpy.array([[203, 239], [718, 515]]))
print(f'{p:E}')

1.055899E-05


# 6

In [51]:
# Не используется, но я просто оставлю это здесь.
def proportions_diff_confint_rel(f: float, g: float, n: int, alpha=0.05) -> (float, float):
    z = stats.norm.ppf(1 - alpha / 2.)
    mu = (f - g) / n
    interval = z * sqrt((f + g) / n ** 2 - (f - g) ** 2 / (n ** 3))
    return (mu - interval, mu + interval)


def proportions_diff_confint_ind(s1: float, n1: float, s2: float, n2: float, alpha=0.05) -> (float, float):
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = s1 / n1
    p2 = s2 / n2
    mu = p1 - p2
    interval = z * sqrt(p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2)
    
    return mu - interval, mu + interval


# …вычитая долю женщин из доли мужчин.
left, right = proportions_diff_confint_ind(239, 239 + 515, 203, 203 + 718)
print(left, '…', right)
print(f'Answer: {left:.4f}')

0.0539052332158 … 0.139221831415
Answer: 0.0539


# 7

In [38]:
def proportions_diff_z_test(z_stat, alternative='two-sided'):
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)


def proportions_diff_z_stat_ind(s1, n1, s2, n2):
    p1 = s1 / n1
    p2 = s2 / n2 
    P = float(p1 * n1 + p2 * n2) / (n1 + n2)
    
    return (p1 - p2) / sqrt(P * (1 - P) * (1. / n1 + 1. / n2))


p = proportions_diff_z_test(proportions_diff_z_stat_ind(203, 203 + 718, 239, 239 + 515), 'two-sided')
print(f'{p:E}')

8.153453E-06


# 8 & 9 & 10

In [45]:
def cramers_stat(observed):
    """
    https://stackoverflow.com/a/38032115/359730
    """
    chi2, *_ = stats.chi2_contingency(observed)
    n = observed.sum()
    return sqrt(chi2 / (n * (min(observed.shape) - 1)))


observed = numpy.array([
    [197, 111, 33],
    [382, 685, 331],
    [110, 342, 333],
])

chi2, p, dof, expected = stats.chi2_contingency(observed)
print(f'Chi2: {chi2:.4f}')
print(f'p: {p:E}')
print(f'V: {cramers_stat(observed):.4f}')

Chi2: 293.6831
p: 2.496430E-62
V: 0.2412
